ssh -p 224 ryo@192.168.1.100

In [34]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities

import time
import json
from datetime import datetime

import requests
from bs4 import BeautifulSoup
from key import sbi_id, sbi_pass


In [38]:
caps = DesiredCapabilities.CHROME
caps['goog:loggingPrefs'] = {'performance': 'ALL'}

options = Options()

options.add_experimental_option("detach", True)

options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36")

In [39]:
driver = webdriver.Chrome(options=options)

driver.get("https://www.sbisec.co.jp/ETGate")

id_input = driver.find_element(By.NAME, "user_id")
# time.sleep(1)
id_input.send_keys(sbi_id)

pass_input = driver.find_element(By.NAME, "user_password")
# time.sleep(1)
pass_input.send_keys(sbi_pass)

driver.find_element(By.NAME, "ACT_login").click()

In [40]:
driver.get("https://global.sbisec.co.jp/home")

In [41]:
driver.find_element(By.XPATH, r'//*[@id="root"]/main/article/div[1]/div[2]/a[2]').click()

In [37]:
# print(original_window)
# print(new_window)
print(driver.window_handles)

['1F2D6582AA2131F3AFAD9E18500C03BB']


In [56]:
# input_element = driver.find_element(By.CLASS_NAME, "input-sm")
# input_element.send_keys("エヌビディア")
# time.sleep(0.3)

In [42]:
stock_name = "TSLA"
driver.get(f"https://global.sbisec.co.jp/invest/us/stock/{stock_name}?resource=news&searchType=include")
time.sleep(0.3)

In [57]:
# search_button = driver.find_element(By.CSS_SELECTOR, "button[data-ga-button='brandsearch']")

# search_button.click()
# time.sleep(0.3)

In [58]:
# 'ニュース'タブに対応するボタンを見つける
# news_tab = driver.find_element(By.CSS_SELECTOR, "button[data-ga-tab='news']")
# ボタンをクリックする
# news_tab.click()
# time.sleep(0.3)

In [43]:
iframe = driver.page_source

soup = BeautifulSoup(iframe, "html.parser")
url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"

# Search for iframes or similar tags that might contain the specified URL pattern
iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

# Display the found URLs from iframes
iframe_urls

['https://graph.sbisec.co.jp/sbinews/pc?symbol=TSLA&token=2B19097C46A7701E96D35A6ECDAD7E72B59CFE5C62EC6BBD96006851B042687954B7F7378D9A0351008598FC10860A8925DE272521C04C2D30A1BD64194E818247258D4E8EA28D2D1F696DFF5062AE40']

In [44]:
print(iframe_urls[0])
driver.execute_script(f"window.open('{iframe_urls[0]}', '_blank');")

https://graph.sbisec.co.jp/sbinews/pc?symbol=TSLA&token=2B19097C46A7701E96D35A6ECDAD7E72B59CFE5C62EC6BBD96006851B042687954B7F7378D9A0351008598FC10860A8925DE272521C04C2D30A1BD64194E818247258D4E8EA28D2D1F696DFF5062AE40


In [95]:
logs = driver.get_log('performance')
# logs

In [98]:
# クッキーの取得
cookies = driver.get_cookies()

print(cookies)
# 特定のクッキー（token）を取得
token = None
for cookie in cookies:
    if cookie['name'] == 'token':
        token = cookie['value']
        break

# 結果を表示
if token:
    print("Token:", token)
else:
    print("Token not found")


[{'domain': '.sbisec.co.jp', 'expiry': 1755069993, 'httpOnly': False, 'name': '__smn_fid', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'nRaVsZ9163icefefT1kBGgiIugUQn-yGs4kyQPT6rfi13PMT'}, {'domain': '.sbisec.co.jp', 'expiry': 1755069963, 'httpOnly': False, 'name': '_kyp', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'QEkQBpnMfhyHlBi4PT/SxNVHatBsAYJc+i4nqrGoGnKXISzFOgpglz6LieoObQ8kE11OSwboScIh4iSKIsjA_.sbisec.co.jp+eh+global.sbisec.co.jp+guid+fb88da93c0ea05637cc3143e9d5f9a92b1816856addddfd87444d489309cdaa1'}, {'domain': '.sbisec.co.jp', 'expiry': 1755069963, 'httpOnly': False, 'name': '_ga_2X743H1G05', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': 'GS1.1.1720509904.1.1.1720509963.1.0.0'}, {'domain': '.sbisec.co.jp', 'expiry': 1754205963, 'httpOnly': False, 'name': '_uetvid', 'path': '/', 'sameSite': 'Lax', 'secure': False, 'value': '5c4175103dc411ef8584a110b4730537|1g5j4b3|1720509963387|7|1|bat.bing.com/p/insights/c/x'}, {'domain': '.sbisec.co.jp',

In [103]:
token = iframe_urls[0].split("token=")[1]
url = f"https://graph.sbisec.co.jp/sbinews/srvdetail?symbol={stock_name}&token={token}"

res = requests.get(url)
data = res.json()
path = "test.json"
with open(path, 'w', encoding='utf-8') as json_file:
    json.dump(data, json_file, ensure_ascii=False, indent=4)

In [73]:
def process_browser_log_entry(entry):
    response = json.loads(entry['message'])['message']
    return response

# 'message'フィールドを処理してリクエストイベントを取得
events = [process_browser_log_entry(entry) for entry in logs]
events = [event for event in events if 'Network.requestWillBeSent' in event['method']]

detected_urls = []
for item in events:
    if "request" in item["params"]:
        if "documentURL" in item["params"]["request"]:
            url = item["params"]["request"]["documentURL"]
            if "https://graph.sbisec.co.jp/sbinews/srvdetail?symbol=" in url:
                detected_urls.append(url)

print("Detected URLs with 'symbol=':")
for url in detected_urls:
    print(url)

Detected URLs with 'symbol=':


In [10]:
print(driver.window_handles)

['D9A7B2D9B8DE9141180C9D8DA7A7F6E8', '8D468376BDE93380BC20E5A374D5EBB0']


In [45]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
# new_window = [window for window in driver.window_handles if window != original_window][1]
new_window = driver.window_handles[1]
driver.switch_to.window(new_window)
time.sleep(0.5)

In [12]:
print(original_window)
print(new_window)
print(driver.window_handles)

D9A7B2D9B8DE9141180C9D8DA7A7F6E8
8D468376BDE93380BC20E5A374D5EBB0
['D9A7B2D9B8DE9141180C9D8DA7A7F6E8', '8D468376BDE93380BC20E5A374D5EBB0']


In [62]:
prev_to_next_button = driver.find_element(By.CSS_SELECTOR, "li.paginationjs-page.paginationjs-last.J-paginationjs-page a")
prev_to_next_button.click()


In [25]:
# print(driver.page_source)

In [15]:
netLog = driver.get_log("performance")

InvalidArgumentException: Message: invalid argument: log type 'performance' not found
  (Session info: chrome=126.0.6478.127)
Stacktrace:
	GetHandleVerifier [0x00007FF72B2EEEA2+31554]
	(No symbol) [0x00007FF72B267ED9]
	(No symbol) [0x00007FF72B12872A]
	(No symbol) [0x00007FF72B1B2760]
	(No symbol) [0x00007FF72B19CFF0]
	(No symbol) [0x00007FF72B16A33B]
	(No symbol) [0x00007FF72B16AED1]
	GetHandleVerifier [0x00007FF72B5F8B1D+3217341]
	GetHandleVerifier [0x00007FF72B645AE3+3532675]
	GetHandleVerifier [0x00007FF72B63B0E0+3489152]
	GetHandleVerifier [0x00007FF72B39E776+750614]
	(No symbol) [0x00007FF72B27375F]
	(No symbol) [0x00007FF72B26EB14]
	(No symbol) [0x00007FF72B26ECA2]
	(No symbol) [0x00007FF72B25E16F]
	BaseThreadInitThunk [0x00007FFBB658257D+29]
	RtlUserThreadStart [0x00007FFBB864AF28+40]


In [46]:
def collect_news_data(driver, wait):
    try:
        news_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "data-row")))
        news_data = []
        for item in reversed(news_items):
            item.click()
            wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "content")))  # コンテンツが表示されるのを待つ
            headline = item.find_element(By.CLASS_NAME, "headline").text.strip()
            date_time = item.find_element(By.CLASS_NAME, "date").text.strip()
            content = item.find_element(By.CLASS_NAME, "content").text.strip() if item.get_attribute("is-open") == "true" else "コンテンツが利用不可"
            news_data.append({"headline": headline, "date": date_time, "content": content})
        return news_data
    except Exception as e:
        print(f"Error while collecting news data: {e}")
        return []

def navigate_and_collect(driver):
    wait = WebDriverWait(driver, 10)
    all_news_data = []

    while True:
        new_data = collect_news_data(driver, wait)
        all_news_data.extend(new_data)

        try:
            current_page = int(wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".paginationjs-page.active a"))).text)
            if current_page == 1:
                break
            prev_button = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, ".paginationjs-prev:not(.disabled) a")))
            prev_button.click()
            wait.until(EC.staleness_of(prev_button))  # ページが完全に遷移するのを待つ
        except TimeoutException:
            print("Waiting for the previous button to be clickable timed out. Trying again...")
            continue  # エラーが出た場合、同じページで再試行する
        except NoSuchElementException:
            print("Previous button not found, possibly on the first page.")
            break

    return all_news_data

collected_data = navigate_and_collect(driver)

# 収集したデータをJSONファイルに保存
today_date = datetime.now().strftime("%Y-%m-%d")
filename = f"./scrape_data/{today_date}_test_data.json"
with open(filename, 'w', encoding='utf-8') as f:
    json.dump(collected_data, f, ensure_ascii=False, indent=4)

Error while collecting news data: Message: 

Waiting for the previous button to be clickable timed out. Trying again...
Error while collecting news data: Message: 

Waiting for the previous button to be clickable timed out. Trying again...
Error while collecting news data: Message: 

Waiting for the previous button to be clickable timed out. Trying again...


KeyboardInterrupt: 

In [27]:
print(driver.window_handles)

['301EB73D6654F955565F58AB50933385', 'C33940D93B42C48258FA6AC5928810EC', '577BA6E8CA2C7E412F18882CD9E188EA']


In [ ]:

driver.close()
driver.switch_to.window(original_window)

driver.find_element(By.ID, "logo-icon").click()

In [ ]:
def load_symbols(filename):
    with open(filename, 'r') as file:
        data = json.load(file)
    symbols = []
    for entry in data:
        symbols.append(entry["symbol"])
    return symbols

In [81]:

def news(n_stock):
    input_element = driver.find_element(By.CLASS_NAME, "input-sm")
    input_element.send_keys(n_stock)
    time.sleep(0.3)

    search_button = driver.find_element(By.CSS_SELECTOR, "button[data-ga-button='brandsearch']")

    search_button.click()
    time.sleep(0.3)

    # 'ニュース'タブに対応するボタンを見つける
    news_tab = driver.find_element(By.CSS_SELECTOR, "button[data-ga-tab='news']")

    # ボタンをクリックする
    news_tab.click()
    time.sleep(0.3)

    iframe = driver.page_source

    soup = BeautifulSoup(iframe, "html.parser")
    url_pattern = "https://graph.sbisec.co.jp/sbinews/pc?"

    # Search for iframes or similar tags that might contain the specified URL pattern
    iframe_urls = [iframe.get('src') for iframe in soup.find_all('iframe', src=True) if iframe['src'].startswith(url_pattern)]

    driver.execute_script(f"window.open('{iframe_urls[0]}', '_blank');")

    # 現在のウィンドウハンドルを取得
    original_window = driver.current_window_handle

    # 新しいタブまたはウィンドウに切り替え
    new_window = [window for window in driver.window_handles if window != original_window][1]
    driver.switch_to.window(new_window)
    time.sleep(0.5)

    prev_to_next_button = driver.find_element(By.CSS_SELECTOR, "li.paginationjs-page.paginationjs-last.J-paginationjs-page a")
    prev_to_next_button.click()

    # 今日の日付を取得してファイル名に使用
    today_date = datetime.now().strftime("%Y-%m-%d")
    filename = f"./scrape_data/{n_stock}_{today_date}_news_data.json"

    # WebDriverWaitの設定
    wait = WebDriverWait(driver, 10)

    news_items = wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME, "data-row")))
    news_data = []

    for item in reversed(news_items):
        # ニュース項目をクリック
        wait.until(EC.element_to_be_clickable(item)).click()

        # コンテンツが表示されるのを待つ
        content = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, ".data-row[is-open='true'] .content")))
        # time.sleep(0.02)  # 展開待ち

        headline = item.find_element(By.CLASS_NAME, "headline").text.strip()
        date_time = item.find_element(By.CLASS_NAME, "date").text.strip()
        content_text = content.text.strip() if content.text.strip() else "No content available"

        news_data.append({"headline": headline, "date": date_time, "content": content_text})


    # 初回のニュースデータ収集
    all_news_data = news_data
    print(all_news_data)

    # 「Previous page」の処理
    while True:
        try:
            prev_button = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, "#pagination-container1 .paginationjs-prev:not(.disabled)")))
            if "disabled" in prev_button.get_attribute("class"):
                print("Reached the first page, no more previous pages.")
                break
            prev_button.click()
            wait.until(EC.staleness_of(news_items[0]))
            new_data = collect_news_data()
            all_news_data.extend(new_data)
            print(new_data)
        except:
            break


    # JSONファイルに保存
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(all_news_data, f, ensure_ascii=False, indent=4)

    driver.close()
    driver.switch_to.window(original_window)

    driver.find_element(By.ID, "logo-icon").click()



In [83]:
# 現在のウィンドウハンドルを取得
original_window = driver.current_window_handle

# 新しいタブまたはウィンドウに切り替え
new_window = [window for window in driver.window_handles if window != original_window][0]
driver.switch_to.window(new_window)
time.sleep(0.5)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF76AF01502+60802]
	(No symbol) [0x00007FF76AE7AC02]
	(No symbol) [0x00007FF76AD37CE4]
	(No symbol) [0x00007FF76AD0DFDF]
	(No symbol) [0x00007FF76ADB1E57]
	(No symbol) [0x00007FF76ADB8DDB]
	(No symbol) [0x00007FF76ADAAB40]
	(No symbol) [0x00007FF76AD78FEC]
	(No symbol) [0x00007FF76AD79C21]
	GetHandleVerifier [0x00007FF76B20411D+3217821]
	GetHandleVerifier [0x00007FF76B2460B7+3488055]
	GetHandleVerifier [0x00007FF76B23F03F+3459263]
	GetHandleVerifier [0x00007FF76AFBB846+823494]
	(No symbol) [0x00007FF76AE85F9F]
	(No symbol) [0x00007FF76AE80EC4]
	(No symbol) [0x00007FF76AE81052]
	(No symbol) [0x00007FF76AE718A4]
	BaseThreadInitThunk [0x00007FFE7FE5257D+29]
	RtlUserThreadStart [0x00007FFE807CAA48+40]


In [82]:
news("NVDA")

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.60)
Stacktrace:
	GetHandleVerifier [0x00007FF76AF01502+60802]
	(No symbol) [0x00007FF76AE7AC02]
	(No symbol) [0x00007FF76AD37CE4]
	(No symbol) [0x00007FF76AD0DFDF]
	(No symbol) [0x00007FF76ADB1E57]
	(No symbol) [0x00007FF76ADC98D1]
	(No symbol) [0x00007FF76ADAA923]
	(No symbol) [0x00007FF76AD78FEC]
	(No symbol) [0x00007FF76AD79C21]
	GetHandleVerifier [0x00007FF76B20411D+3217821]
	GetHandleVerifier [0x00007FF76B2460B7+3488055]
	GetHandleVerifier [0x00007FF76B23F03F+3459263]
	GetHandleVerifier [0x00007FF76AFBB846+823494]
	(No symbol) [0x00007FF76AE85F9F]
	(No symbol) [0x00007FF76AE80EC4]
	(No symbol) [0x00007FF76AE81052]
	(No symbol) [0x00007FF76AE718A4]
	BaseThreadInitThunk [0x00007FFE7FE5257D+29]
	RtlUserThreadStart [0x00007FFE807CAA48+40]


In [106]:
# JSONファイルの読み込み
with open('./scrape_data/stock_name.json', 'r', encoding="utf-8") as file:
    data = json.load(file)

# 全てのシンボルをプリント
for category in data:
    for company in data[category]:
        print(company['symbol'])


A
AAL
AAPL
ABBV
ABNB
ABT
ACGL
ACN
ADBE
ADI
ADM
ADP
ADSK
AEE
AEP
AES
AFL
AIG
AIZ
AJG
AKAM
ALB
ALGN
ALL
ALLE
AMAT
AMCR
AMD
AME
AMGN
AMP
AMT
AMZN
ANET
ANSS
AON
AOS
APA
APD
APH
APTV
ARE
ATO
AVB
AVGO
AVY
AWK
AXON
AXP
AZO
BA
BAC
BALL
BAX
BBWI
BBY
BDX
BEN
BF-B
BG
BIIB
BIO
BK
BKNG
BKR
BLDR
BLK
BMY
BR
BRK-B
BRO
BSX
BWA
BX
BXP
C
CAG
CAH
CARR
CAT
CB
CBOE
CBRE
CCI
CCL
CDNS
CDW
CE
CEG
CF
CFG
CHD
CHRW
CHTR
CI
CINF
CL
CLX
CMA
CMCSA
CME
CMG
CMI
CMS
CNC
CNP
COF
COO
COP
COR
COST
CPAY
CPB
CPRT
CPT
CRL
CRM
CSCO
CSGP
CSX
CTAS
CTLT
CTRA
CTSH
CTVA
CVS
CVX
CZR
D
DAL
DAY
DD
DE
DECK
DFS
DG
DGX
DHI
DHR
DIS
DLR
DLTR
DOC
DOV
DOW
DPZ
DRI
DTE
DUK
DVA
DVN
DXCM
EA
EBAY
ECL
ED
EFX
EG
EIX
EL
ELV
EMN
EMR
ENPH
EOG
EPAM
EQIX
EQR
EQT
ES
ESS
ETN
ETR
ETSY
EVRG
EW
EXC
EXPD
EXPE
EXR
F
FANG
FAST
FCX
FDS
FDX
FE
FFIV
FI
FICO
FIS
FITB
FMC
FOX
FOXA
FRT
FSLR
FTNT
FTV
GD
GE
GEHC
GEN
GEV
GILD
GIS
GL
GLW
GM
GNRC
GOOG
GOOGL
GPC
GPN
GRMN
GS
GWW
HAL
HAS
HBAN
HCA
HD
HES
HIG
HII
HLT
HOLX
HON
HPE
HPQ
HRL
HSIC
HST
HSY
HUBB
HUM
HWM
IBM
ICE
IDXX